using psycopg2 todo:
Problem statement :

Set up a simple Library Management System. The system will track books, authors, and book loans. You'll be required to create tables, insert data from the Excel file and perform various queries to manage and analyze the data.

Tasks
a. Create Tables
Create the following tables in your PostgreSQL database:

authors with columns author_id, name, and date_of_birth.
books with columns book_id, title, author_id, publication_year, and genre.
loans with columns loan_id, book_id, loan_date, and return_date.
b. Insert Data
Insert data from the provided Excel sheet into the authors, books, and loans tables.

Use the pandas library to read the CSV files.
Insert the data into the corresponding tables.
c. Perform Queries
Write Python functions to perform the following queries using psycopg2:

Select all books with their authors' names. Use a JOIN to combine books and authors tables.
Select books that have not been returned yet. Use the WHERE clause to filter books where return_date is NULL.
Group by genre and count the number of books in each genre. Use the GROUP BY clause to aggregate data by genre.
Order books by publication year in ascending order. Use the ORDER BY clause to sort books.
Select authors who were born before 1900. Use the WHERE clause to filter authors based on their date_of_birth.
Update the return date of a specific loan. Use the UPDATE statement to modify the return_date in the loans table.
Delete a book from the library by its book_id. Ensure to handle foreign key constraints if applicable.

In [1]:
import psycopg2
import configparser

CONNECTDICT = None

def setConnectionstr():
    global CONNECTDICT
    CONNECTDICT = {}
    config = configparser.ConfigParser()
    config.read('db_connection.conf')
    for option in config['postgres']:
        CONNECTDICT[option] = config.get('postgres', option)
setConnectionstr()
CONNECTDICT

{'host': 'localhost', 'user': 'postgres', 'password': '1q2w3e', 'port': '5433'}

In [2]:
try:
    conn = psycopg2.connect(**CONNECTDICT)
    cur = conn.cursor()
    # Create authors table
    cur.execute("""
        CREATE TABLE authors (
            author_id SERIAL PRIMARY KEY,
            name VARCHAR(255) NOT NULL,
            date_of_birth DATE NOT NULL
        )
    """)
    
    # Create books table
    cur.execute("""
        CREATE TABLE books (
            book_id SERIAL PRIMARY KEY,
            title VARCHAR(255) NOT NULL,
            author_id INTEGER NOT NULL,
            publication_year INTEGER NOT NULL,
            genre VARCHAR(100),
            FOREIGN KEY (author_id)
                REFERENCES authors (author_id)
                ON DELETE CASCADE
        )
    """)
    
    # Create loans table
    cur.execute("""
        CREATE TABLE loans (
            loan_id SERIAL PRIMARY KEY,
            book_id INTEGER NOT NULL,
            loan_date DATE NOT NULL,
            return_date DATE,
            FOREIGN KEY (book_id)
                REFERENCES books (book_id)
                ON DELETE CASCADE
        )
    """)

    cur.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print("Catch exception: ", error)
finally:
    if conn is not None:
        conn.close()

In [8]:
import pandas as pd
import openpyxl

try:
    conn = psycopg2.connect(**CONNECTDICT)
    cur = conn.cursor()
    
    # Read data from Excel files
    authors_df = pd.read_excel('assignment3.xlsx', 'authors')
    books_df = pd.read_excel('assignment3.xlsx', 'books')
    loans_df = pd.read_excel('assignment3.xlsx', 'loans')
    
    # Insert data into authors table
    author_id_map = {}
    for i, row in authors_df.iterrows():
        cur.execute("INSERT INTO authors (name, date_of_birth) VALUES (%s, %s) RETURNING author_id", (row['name'], row['date_of_birth']))
        author_id = cur.fetchone()[0]
        author_id_map[row['author_id']] = author_id
    
    # Insert data into books table
    book_id_map = {}
    for i, row in books_df.iterrows():
        cur.execute("INSERT INTO books (title, author_id, publication_year, genre) VALUES (%s, %s, %s, %s) RETURNING book_id", 
                    (row['title'], author_id_map[row['author_id']], row['publication_year'], row['genre']))
        book_id = cur.fetchone()[0]
        book_id_map[row['book_id']] = book_id
    
    # Insert data into loans table
    for i, row in loans_df.iterrows():
        # Handle NaT values by replacing them with None
        return_date = row['return_date'] if pd.notna(row['return_date']) else None
        cur.execute("INSERT INTO loans (book_id, loan_date, return_date) VALUES (%s, %s, %s)", 
                    (book_id_map[row['book_id']], row['loan_date'], return_date))
    
    cur.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print("Catch exception: ", error)
finally:
    if conn is not None:
        conn.close()

In [9]:
try:
    conn = psycopg2.connect(**CONNECTDICT)
    cur = conn.cursor()
    
    cur.execute("""
        SELECT books.title, authors.name
        FROM books
        JOIN authors ON books.author_id = authors.author_id
    """)
    books_with_authors = cur.fetchall()
    print("Books with Authors:")
    for row in books_with_authors:
        print(row)
    
    cur.execute("SELECT * FROM loans WHERE return_date IS NULL")
    books_not_returned = cur.fetchall()
    print("\nBooks Not Returned:")
    for row in books_not_returned:
        print(row)
    
    cur.execute("""
        SELECT genre, COUNT(*)
        FROM books
        GROUP BY genre
    """)
    books_by_genre = cur.fetchall()
    print("\nBooks by Genre:")
    for row in books_by_genre:
        print(row)

    cur.execute("SELECT * FROM books ORDER BY publication_year ASC")
    books_ordered_by_year = cur.fetchall()
    print("\nBooks Ordered by Publication Year:")
    for row in books_ordered_by_year:
        print(row)
  
    cur.execute("SELECT * FROM authors WHERE date_of_birth < '1900-01-01'")
    authors_born_before_1900 = cur.fetchall()
    print("\nAuthors Born Before 1900:")
    for row in authors_born_before_1900:
        print(row)
    
    loan_id_to_update = 1  
    new_return_date = '2024-08-01'
    cur.execute("UPDATE loans SET return_date = %s WHERE loan_id = %s", (new_return_date, loan_id_to_update))
    
    book_id_to_delete = 1 
    cur.execute("DELETE FROM books WHERE book_id = %s", (book_id_to_delete,))
    
    cur.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print("Catch exception: ", error)
finally:
    if conn is not None:
        conn.close()

Books with Authors:
('The Great Gatsby', 'F. Scott Fitzgerald')
('To Kill a Mockingbird', 'Harper Lee')
('1984', 'George Orwell')
('Pride and Prejudice', 'Jane Austen')
('The Catcher in the Rye', 'J.D. Salinger')
('The Hobbit', 'J.R.R. Tolkien')
('Brave New World', 'George Orwell')
('Jane Eyre', 'Jane Austen')
('The Lord of the Rings', 'J.R.R. Tolkien')
('Animal Farm', 'George Orwell')

Books Not Returned:
(12, 28, datetime.date(2024, 2, 15), None)
(13, 29, datetime.date(2024, 3, 1), None)
(15, 31, datetime.date(2024, 3, 10), None)

Books by Genre:
('Fantasy', 2)
('Dystopian', 3)
('Romance', 2)
('Fiction', 3)

Books Ordered by Publication Year:
(26, 'Pride and Prejudice', 34, 1813, 'Romance')
(30, 'Jane Eyre', 34, 1847, 'Romance')
(23, 'The Great Gatsby', 31, 1925, 'Fiction')
(29, 'Brave New World', 33, 1932, 'Dystopian')
(28, 'The Hobbit', 36, 1937, 'Fantasy')
(32, 'Animal Farm', 33, 1945, 'Dystopian')
(25, '1984', 33, 1949, 'Dystopian')
(27, 'The Catcher in the Rye', 35, 1951, 'Ficti